<a href="https://colab.research.google.com/github/tara3165/NLP-/blob/CompliantAnalysis/Bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
cd drive

/content/drive


In [24]:
ls

MyDrive/


In [4]:
!pip install BERTopic
!pip install keybert
!pip install keyphrase-vectorizers
!python -m pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 22.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 15.7 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731

In [5]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance


import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
from string import punctuation
import transformers
from bertopic.representation import TextGeneration


In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com','XX','XXXX','XXX','XXXXX','00'
,'xx','xxxx','xxx','xxxxx']
vectorizer_model=KeyphraseCountVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df_complaints=pd.read_csv('../drive/MyDrive/Colab Notebooks/Complaint/complaints.csv',low_memory=False)
desc_col='Consumer complaint narrative'

df_complaints['Year']=pd.to_datetime(df_complaints['Date received']) .dt.year
df_complaints['Month']=pd.to_datetime(df_complaints['Date received']) .dt.month
df_complaints['Day']=pd.to_datetime(df_complaints['Date received']) .dt.day

def clean(text):
  #very light data cleansing
    #remove @ppl, url

    output = re.sub(r'https://\S*','', text)
    output = re.sub(r'@\S*','',output)

    #remove \r, \n
    rep = r'|'.join((r'\r',r'\n'))
    output = re.sub(rep,'',output)

    # rep_covid = r'|'.join((r'\W*covid\S*',r'\W*COVID\S*',r'\W*coronavirus\S*', r'\W*Covid\S*',r'\W*Coronavirus\S*'))
    # output = re.sub(rep_covid,'', output)

    rep_covid = r'|'.join((r'\W*XX\S*',r'\W*XXX\S*',r'\W*XXXX\S*', r'\W*XXXXX\S*',r'\W*XXXXXX\S*',r'\W*00\S*'))
    output = re.sub(rep_covid,'', output)

    #remove duplicated punctuation
    output = re.sub(r'([!()\-{};:,<>./?@#$%\^&*_~]){2,}', lambda x: x.group()[0], output)

    #remove extra space
    output = re.sub(r'\s+', ' ', output).strip()

    #remove string if string only contains punctuation
    if sum([i.isalpha() for i in output])== 0:
        output = ''

    #remove string if length<5
    if len(output.split()) < 5:
        output = ''

    return output#.lower()




First, the amount of datapoint classified as outliers is handled by the min_samples parameters in HDBSCAN. This value is automatically set to the same value of min_cluster_size. However, you can set it independently if you want to reduce the number of generated outliers. Lowering this value will result in less noise being generated.




In [8]:
from bertopic.representation._base import BaseRepresentation
import typing
from typing import *
from tqdm import tqdm
from transformers import pipeline

class CustomRepresentationModel(BaseRepresentation):
    def __init__(self,
                 model: Union[str, pipeline],
                 prompt: str = None,
                 pipeline_kwargs: Mapping[str, Any] = {},
                 random_state: int = 42,
                 nr_docs: int = 4,
                 diversity: float = None,
                 doc_length: int = None,
                 tokenizer: Union[str, Callable] = None,
                 task: str='text-generation',
                 max_new_tokens: int=10
                 ):
        transformers.set_seed(random_state)
        # if isinstance(model, str):
        #     self.model = pipeline("text-generation", model=model)
        # elif isinstance(model, Pipeline):
        #     self.model = model
        # else:
        #     raise ValueError("Make sure that the HF model that you"
        #                      "pass is either a string referring to a"
        #                      "HF model or a `transformers.pipeline` object.")

        if isinstance(model, str):
            self.model = pipeline("text-generation", model=model)
        else :
            self.model = transformers.pipeline(
                        model=model, tokenizer=tokenizer,
                        task=task,
                        temperature=0.1,
                        max_new_tokens=max_new_tokens,
                        repetition_penalty=1.1
                    )

        self.DEFAULT_PROMPT=""
        self.prompt = prompt if prompt is not None else self.DEFAULT_PROMPT
        self.default_prompt_ = self.DEFAULT_PROMPT
        self.pipeline_kwargs = pipeline_kwargs
        self.nr_docs = nr_docs
        self.diversity = diversity
        self.doc_length = doc_length
        self.tokenizer = tokenizer
        self.prompts_ = []


    def _truncate_document(self,topic_model, doc_length, tokenizer, document: str):
        """ Truncate a document to a certain length

        If you want to add a custom tokenizer, then it will need to have a `decode` and
        `encode` method. An example would be the following custom tokenizer:

        ```python
        class Tokenizer:
            'A custom tokenizer that splits on commas'
            def encode(self, doc):
                return doc.split(",")

            def decode(self, doc_chuncks):
                return ",".join(doc_chuncks)
        ```

        You can use this tokenizer by passing it to the `tokenizer` parameter.

        Arguments:
            topic_model: A BERTopic model
            doc_length: The maximum length of each document. If a document is longer,
                        it will be truncated. If None, the entire document is passed.
            tokenizer: The tokenizer used to calculate to split the document into segments
                      used to count the length of a document.
                          * If tokenizer is 'char', then the document is split up
                            into characters which are counted to adhere to `doc_length`
                          * If tokenizer is 'whitespace', the document is split up
                            into words separated by whitespaces. These words are counted
                            and truncated depending on `doc_length`
                          * If tokenizer is 'vectorizer', then the internal CountVectorizer
                            is used to tokenize the document. These tokens are counted
                            and trunctated depending on `doc_length`. They are decoded with
                            whitespaces.
                          * If tokenizer is a callable, then that callable is used to tokenize
                            the document. These tokens are counted and truncated depending
                            on `doc_length`
            document: A single document

        Returns:
            truncated_document: A truncated document
        """
        if doc_length is not None:
            if tokenizer == "char":
                truncated_document = document[:doc_length]
            elif tokenizer == "whitespace":
                truncated_document = " ".join(document.split()[:doc_length])
            elif tokenizer == "vectorizer":
                tokenizer = topic_model.vectorizer_model.build_tokenizer()
                truncated_document = " ".join(tokenizer(document)[:doc_length])
            elif hasattr(tokenizer, 'encode') and hasattr(tokenizer, 'decode'):
                encoded_document = tokenizer.encode(document)
                truncated_document = tokenizer.decode(encoded_document[:doc_length])
            return truncated_document
        #print(document)
        return document

    def extract_topics(self, topic_model, documents, c_tf_idf, topics
                      ) -> Mapping[str, List[Tuple[str, float]]]:
        """ Extract topics

        Arguments:
            topic_model: The BERTopic model
            documents: A dataframe of documents with their related topics
            c_tf_idf: The c-TF-IDF matrix
            topics: The candidate topics as calculated with c-TF-IDF

        Returns:
            updated_topics: Updated topic representations
        """

        # Extract the top 4 representative documents per topic
        #if self.prompt != DEFAULT_PROMPT and "[DOCUMENTS]" in self.prompt:
        repr_docs_mappings, _, _, _ = topic_model._extract_representative_docs(
                c_tf_idf,
                documents,
                topics,
                500,
                self.nr_docs,
                self.diversity
            )
        #else:

        #repr_docs_mappings = {topic: None for topic in topics.keys()}



        updated_topics = {}
        for topic, docs in tqdm(repr_docs_mappings.items(), disable=not topic_model.verbose):
            # Prepare prompt
            truncated_docs = [self._truncate_document(topic_model, self.doc_length, self.tokenizer, doc) for doc in docs]
            prompt = self._create_prompt(truncated_docs, topic, topics)
            self.prompts_.append(prompt)

            # Extract result from generator and use that as label
            topic_description = self.model(prompt, **self.pipeline_kwargs)

            topic_description = [(description["generated_text"].replace(prompt, ""), 1) for description in topic_description]


            # if len(topic_description) < 10:
            #     topic_description += [("", 0) for _ in range(10-len(topic_description))]


            updated_topics[topic] = topic_description[0]

        return updated_topics

    def _create_prompt(self, docs, topic, topics):

        keywords = ", ".join(list(zip(*topics[topic]))[0])
        DEFAULT_PROMPT=""
        # Use the default prompt and replace keywords
        if self.prompt == DEFAULT_PROMPT:
            prompt = self.prompt.replace("[KEYWORDS]", keywords)

        # Use a prompt that leverages either keywords or documents in
        # a custom location
        else:
            prompt = self.prompt
            if "[KEYWORDS]" in prompt:
                prompt = prompt.replace("[KEYWORDS]", keywords)
            if "[DOCUMENTS]" in prompt:
                to_replace = ""
                for doc in docs:
                    to_replace += f"- {doc}\n"
                prompt = prompt.replace("[DOCUMENTS]", to_replace)

        return prompt



In [20]:
# we add this to remove stopwords that can pollute topcs
#vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)
vectorizer_model=KeyphraseCountVectorizer(stop_words=stopwords)


# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# umap_model = UMAP(n_neighbors=3, n_components=100, min_dist=0.05)
# hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=5,
#                         gen_min_span_tree=True,
#                         prediction_data=True)

# representation_model = MaximalMarginalRelevance(diversity=0.2)
# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)


# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)
vectorizer_model=KeyphraseCountVectorizer(stop_words=stopwords,pos_pattern="<N.*>+<J.*>*")


embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=5, n_components=100, min_dist=0.05,
                        random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=3,
                        gen_min_span_tree=True,
                        prediction_data=True)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

#cluster_model = KMeans(n_clusters=50)


mmr = MaximalMarginalRelevance(diversity=0.2)
kb_mmr = [KeyBERTInspired(top_n_words=30),
                                MaximalMarginalRelevance(diversity=.5)]

pos_patterns = [
            [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
            [{'POS': 'NOUN'}], [{'POS': 'ADJ'}],
             [{'POS': '<N.*>+<J.*>*'}]
             ]

post = PartOfSpeech("en_core_web_sm", pos_patterns=pos_patterns)

# Create your representation model in gpt2
 # Llama 2 Tokenizer
model_id='gpt2'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True

)
p=TextGeneration('gpt2').default_prompt_

#p="I have a topic described by the following keywords: [KEYWORDS]. The name of this topic is "
# p=f"""
# I have a topic described by the following keywords: [KEYWORDS].
# The name of this topic is:
# """
gptrep =CustomRepresentationModel(model=model,tokenizer=tokenizer,prompt=p,task='text-generation',max_new_tokens=10)


####
prompt = "\n I have a topic described by the following keywords: [KEYWORDS].\nBased on the previous keywords, this topic is about "

# Create your representation model
# generator = transformers.pipeline('text2text-generation', model='google/flan-t5-base')
# flnrep= TextGeneration(model=generator,prompt=prompt, tokenizer = generator.tokenizer, doc_length=510)
model_id='google/flan-t5-base'
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

flnrep=CustomRepresentationModel(model=model,tokenizer=tokenizer,prompt=prompt,task='text2text-generation',max_new_tokens=10)
#
representation_model={
    'mmr':mmr,
    'kb_mmr':kb_mmr,
    'post':post#,
   # 'gptrep':gptrep,
   # 'flnrep':flnrep

}





In [21]:
topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=10,
    language='english',
    calculate_probabilities=True,
    verbose=True,
    nr_topics='auto',
    ctfidf_model=ctfidf_model,
    representation_model=representation_model
)
df_complaints=df_complaints[df_complaints['Year']==2024]
data=df_complaints[df_complaints['Year']==2024]
data=data.sample(2000)
#data=data[data['Month']==1]
#data=data[data['Day'].isin([1,2,3,4])]
data[desc_col]=data[desc_col].apply(clean)
data=list(data[desc_col])
len(data)

2000

Second, after training our BERTopic model, we can assign outliers to topics by making use of the .reduce_outliers function in BERTopic. An advantage of using this approach is that there are four built in strategies one can choose for reducing outliers. Moreover, this technique allows the user to experiment with reducing outliers across a number of strategies and parameters without actually having to re-train the topic model each time. You can learn more about the .reduce_outlier function here. The following is a minimal example of how to use this function:

In [22]:
topics, probs = topic_model.fit_transform(data)
# Reduce outliers
#new_topics = model.reduce_outliers(docs, topics, probabilities=probs, strategy="probabilities")


2024-03-21 06:41:51,934 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2024-03-21 06:46:50,732 - BERTopic - Embedding - Completed ✓
2024-03-21 06:46:50,736 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-21 06:47:02,588 - BERTopic - Dimensionality - Completed ✓
2024-03-21 06:47:02,591 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-21 06:47:03,739 - BERTopic - Cluster - Completed ✓
2024-03-21 06:47:03,741 - BERTopic - Representation - Extracting topics from clusters using representation models.
/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)
2024-03-21 06:50:16,980 - BERTopic - Representation - Completed ✓
2024-03-21 06:50:16,982 - BERTopic - Topic reduction - Reducing number of topics
/usr/local/lib/python3.10/dist-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)
2024-03-21 06:52:31,986 - 

Third, we can replace HDBSCAN with any other clustering algorithm that we want. So we can choose a clustering algorithm, like k-Means, that does not produce any outliers at all. Using k-Means instead of HDBSCAN is straightforward:



In [23]:
# Method 3 - pickle
topic_model.save('../drive/MyDrive/Colab Notebooks/Complaint/topic_model.csv', serialization="pickle")

2024-03-21 06:52:44,615 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
# cluster_model = KMeans(n_clusters=50)
# topic_model = BERTopic(hdbscan_model=cluster_model)

In [63]:
new_topics = topic_model.reduce_outliers(list(data), topics, probabilities=probs, strategy="probabilities")
l=pd.DataFrame({'a':new_topics,'b':topics})
l.to_csv('../drive/MyDrive/Colab Notebooks/Complaint/l.csv')


In [24]:
x=topic_info=topic_model.get_topic_info()

try:
  print(list(x['flnrep']))
  print(set(x['gptrep']))
except:
    pass

In [25]:
topic_info

,Topic,Count,Name,Representation,mmr,kb_mmr,post,Representative_Docs
0,-1,373,-1_delete_account_please_credit,"[delete, account, please, credit, card, bank, ...","[delete, account, please, credit, card, bank, ...","[my credit report, original creditor, consumer...","[account, credit, card, bank, color, usc, copy...",[On I concluded a lease trade-in for my accoun...
1,0,1119,0_consumer_report_information_credit,"[consumer, report, information, credit, report...","[consumer, report, information, credit, report...","[fair credit reporting act, my credit report, ...","[consumer, report, information, credit, report...",[According to the Fair Credit Reporting Act se...
2,1,408,1_wells_fargo_wells fargo_bank,"[wells, fargo, wells fargo, bank, account, car...","[wells, fargo, wells fargo, bank, account, car...","[fraud, refund, fargo, complaint, charges, che...","[fargo, bank, account, card, money, back, frau...",[On the we noticed a large drop in our balance...
3,2,52,2_experian_lie_complaint_fraudulent company,"[experian, lie, complaint, fraudulent company,...","[experian, lie, complaint, fraudulent company,...","[credit reporting act experian, others complai...","[experian, lie, complaint, fraudulent company,...",[I am writing to address a critical matter con...
4,3,25,3_on time_update_payments_time payments,"[on time, update, payments, time payments, my ...","[on time, update, payments, time payments, my ...","[late payment, account incorrect name, synchro...","[payments, sleep, incorrect, history, stress, ...",[Please update my account and remove the inacc...
5,4,23,4_interest_balance_statement_balance transfer,"[interest, balance, statement, balance transfe...","[interest, balance, statement, balance transfe...","[interest payment, credit card statement, paym...","[interest, balance, statement, pay, rate, mont...",[Synchrony Bank intentionally uses misleading ...


In [47]:
topic_info=topic_model.get_topic_info()

# Create your representation model in gpt2
 # Llama 2 Tokenizer
model_id='gpt2'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True

)
prompt=TextGeneration('gpt2').default_prompt_


task='text-generation'
max_new_tokens=5

pip = transformers.pipeline(
                        model=model, tokenizer=tokenizer,
                        task=task,
                        temperature=0.1,
                        max_new_tokens=max_new_tokens,
                        repetition_penalty=1.1 ,
                        penalty_alpha=0.6,
                        top_k=4
                    )

print('**')
for kb_mmr in topic_info['kb_mmr'] :

    s=', '.join(kb_mmr)
    p = prompt.replace("[KEYWORDS]",  s)
    print(s)
    # Extract result from generator and use that as label
    topic_description = pip(p )
    topic_description = [(description["generated_text"].replace(p, ""), 1) for description in topic_description]
    print(topic_description)
    print()

#------------------


print('________________________')


####
prompt = "\n I have a topic described by the following keywords: [KEYWORDS].\nBased on the previous keywords, this topic is about "

# Create your representation model
# generator = transformers.pipeline('text2text-generation', model='google/flan-t5-base')
# flnrep= TextGeneration(model=generator,prompt=prompt, tokenizer = generator.tokenizer, doc_length=510)

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


task='text2text-generation'
max_new_tokens=10

pip = transformers.pipeline(
                        model=model, tokenizer=tokenizer,
                        task=task,
                        temperature=0.1,
                        max_new_tokens=max_new_tokens,
                        repetition_penalty=1.1 ,
                        penalty_alpha=0.3,
                        top_k=4
                    )

print('**')
for kb_mmr in topic_info['kb_mmr'] :

    s=', '.join(kb_mmr)
    p = prompt.replace("[KEYWORDS]",  s)
    print(p)
    # Extract result from generator and use that as label
    topic_description = pip(p )
    topic_description = [(description["generated_text"].replace(p, ""), 1) for description in topic_description]
    print(topic_description)
    print()




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**
my credit report, original creditor, consumer reporting agency, account immediately, us bank, fraudulent, transactions, charges, reporting, violation


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[('My Credit Report and Account', 1)]

fair credit reporting act, my credit report, reporting agency, consumer reporting, identity theft, privacy, agencies, violations, accounts, my rights


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

[('"Fraudulent Payment', 1)]

credit reporting act experian, others complaint experian, experian fixed nothing, fraudulent fraudulent fraudulent fraudulent fraudulent fraudulent, complaint, phone number gor experian, creditors, predatory company still, bureau, investigation


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[('"credit reporting acts".', 1)]

late payment, account incorrect name, synchrony bank, delays, debit card, mortgage holder, incorrect nj pa, immediate action, online, fail


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[('Late Payment - A short', 1)]

interest payment, credit card statement, payment due date, statement balance, transaction fee, card balance, cash advance, charges, balance transfers, citibank
[('Interest Payment and Credit Card', 1)]

________________________
**

 I have a topic described by the following keywords: my credit report, original creditor, consumer reporting agency, account immediately, us bank, fraudulent, transactions, charges, reporting, violation.
Based on the previous keywords, this topic is about 
[('credit report - consumer reporting agency', 1)]


 I have a topic described by the following keywords: fair credit reporting act, my credit report, reporting agency, consumer reporting, identity theft, privacy, agencies, violations, accounts, my rights.
Based on the previous keywords, this topic is about 
[('credit reporting agency', 1)]


 I have a topic described by the following keywords: fraud, refund, fargo, complaint, charges, checking account, citibank, department

In [49]:
x.to_csv('../drive/MyDrive/Colab Notebooks/Complaint/x.csv')

In [ ]:
model.get_document_info(data)

In [22]:
model.get_document_info(data).to_csv('../drive/MyDrive/Colab Notebooks/Complaint/res1.csv')

In [ ]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

cv = topic_model.vectorizer_model
X = cv.fit_transform(data)
doc_tokens =  cv.inverse_transform(X)

id2word = corpora.Dictionary(doc_tokens)
texts = doc_tokens
corpus = [id2word.doc2bow(text) for text in texts]

topic_words = []
for i in range(len(topic_model.get_topic_freq())-1):
  interim = []
  interim = [t[0] for t in model.get_topic(i)]
  topic_words.append(interim)



coherence_model = CoherenceModel(topics=topic_words,
                                 texts=texts,
                                 corpus=corpus,
                                 dictionary=id2word,
                                 coherence='c_v')
coherence_model.get_coherence()

In [39]:
freq={}
for doc in doc_tokens:
   for token in doc:
    if token in freq:
        freq[token]=freq[token]+1
    else:
        freq[token]=1


In [40]:
df_token_frequency=pd.DataFrame(freq.items(), columns=['token', 'frequency'])
#df_token_frequency.plot.bar(x='token', y='frequency', rot=0)

In [37]:
df_token_frequency.sort_values('frequency',ascending=False)
df_token_frequency[df_token_frequency['frequency']>100]

,token,frequency


In [41]:
df_token_frequency.to_csv('../drive/MyDrive/Colab Notebooks/Complaint/token_frequency.csv')

In [ ]:
model.vectorizer_model.get_feature_names_out()

array(['balance transfers', 'credit balance refund', 'rates', ...,
       'transaction account', 'affirmation', 'segment true'], dtype=object)

In [ ]:
topic_term_matrix = model.c_tf_idf_
words =model.vectorizer_model.get_feature_names_out()

In [ ]:
# # Extract keywords
# from keybert import KeyBERT
# kw_model = KeyBERT()
# keywords = kw_model.extract_keywords(docs=list(data),vectorizer=vectorizer_model)

# # Create our vocabulary
# vocabulary = [k[0] for keyword in keywords for k in keyword]
# vocabulary = list(set(vocabulary))

In [ ]:
freq={}
doc_tokens=''
df_token_frequency=''

summarize